In [2]:
from classiq import *
from classiq.qmod.symbolic import pi

# Constants for the harmonic oscillator
omega = 1.0   # Frequency
dt = 0.05     # Time step
total_time = 1.0
n_steps = int(total_time / dt)  # Number of steps in the simulation

# Function to calculate the potential and kinetic energy operators in Pauli terms
def get_hamiltonian():
    # Potential Energy Term: H_potential = (Z + Z)^2
    potential_energy = [
        PauliTerm(pauli=[Pauli.Z, Pauli.I], coefficient=omega**2 / 2),
        PauliTerm(pauli=[Pauli.I, Pauli.Z], coefficient=omega**2 / 2)
    ]
    
    # Kinetic Energy Term: H_kinetic = (X + X)^2
    kinetic_energy = [
        PauliTerm(pauli=[Pauli.X, Pauli.I], coefficient=1 / 2),
        PauliTerm(pauli=[Pauli.I, Pauli.X], coefficient=1 / 2)
    ]
    
    return potential_energy + kinetic_energy


# Main Qfunc for time evolution using Trotter-Suzuki decomposition
@qfunc
def main(qba: Output[QArray[QBit]]) -> None:
    allocate(2, qba)  # Allocate 2 qubits (for a simple 2D harmonic oscillator)
    
    # Get the Pauli decomposition of the Hamiltonian (Potential + Kinetic Energy)
    hamiltonian = get_hamiltonian()
    
    # Apply Trotter-Suzuki decomposition to approximate time evolution
    for _ in range(n_steps):
        suzuki_trotter(
            hamiltonian,
            evolution_coefficient=dt,
            order=1,   # First-order Trotterization
            repetitions=1,   # Single repetition per time step
            qbv=qba
        )


# Classical function to set up and save results
@cfunc
def cmain() -> None:
    result = sample({})
    save({"result": result})


# Create the quantum model and synthesize the quantum program
qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)

In [4]:
from classiq import *
from classiq.qmod.symbolic import pi

# Constants for the harmonic oscillator
omega = 1.0   # Frequency
dt = 0.05     # Time step
total_time = 1.0
n_steps = int(total_time / dt)  # Number of steps in the simulation

# Function to calculate the potential and kinetic energy operators in Pauli terms
def get_hamiltonian():
    # Potential Energy Term: H_potential = (Z + Z)^2
    potential_energy = [
        PauliTerm(pauli=[Pauli.Z, Pauli.I], coefficient=omega**2 / 2),
        PauliTerm(pauli=[Pauli.I, Pauli.Z], coefficient=omega**2 / 2)
    ]
    
    # Kinetic Energy Term: H_kinetic = (X + X)^2
    kinetic_energy = [
        PauliTerm(pauli=[Pauli.X, Pauli.I], coefficient=1 / 2),
        PauliTerm(pauli=[Pauli.I, Pauli.X], coefficient=1 / 2)
    ]
    
    return potential_energy + kinetic_energy


# Main Qfunc for time evolution using Trotter-Suzuki decomposition
@qfunc
def main(qba: Output[QArray[QBit]]) -> None:
    allocate(2, qba)  # Allocate 2 qubits (for a simple 2D harmonic oscillator)
    
    # Get the Pauli decomposition of the Hamiltonian (Potential + Kinetic Energy)
    hamiltonian = get_hamiltonian()
    
    # Apply Trotter-Suzuki decomposition to approximate time evolution
    for _ in range(n_steps):
        suzuki_trotter(
            hamiltonian,
            evolution_coefficient=dt,
            order=1,   # First-order Trotterization
            repetitions=1,   # Single repetition per time step
            qbv=qba
        )


# Classical function to set up and save results
@cfunc
def cmain() -> None:
    result = sample({})
    save({"result": result})


# Create the quantum model and synthesize the quantum program
#qmod = create_model(main, classical_execution_function=cmain)
#qprog = synthesize(qmod)
#show(qprog)



#########################################
quantum_model = create_model(main)
quantum_program =  synthesize(quantum_model)

########################################
show(quantum_program)
########################################


########################################
write_qmod(quantum_model, "qsite_challenge_4")
########################################

qasm = QuantumProgram.from_qprog(quantum_program).qasm

# Specify the file path where you want to save the QASM file
file_path = "qsite_challenge_open_challenge.qasm"

# Save the QASM string to a file
with open(file_path, 'w') as file:
    file.write(qasm)

print(f"QASM file saved at: {file_path}")

# Execute the quantum program and output the results
job = execute(quantum_program)
results = job.result()[0].value.parsed_counts
print(results)

QASM file saved at: qsite_challenge_open_challenge.qasm
[{'qba': [0, 0]}: 1301, {'qba': [0, 1]}: 338, {'qba': [1, 0]}: 316, {'qba': [1, 1]}: 93]


In [6]:

from classiq import *

from classiq.interface.backend.backend_preferences import ClassiqBackendPreferences
from classiq.interface.executor.execution_preferences import ExecutionPreferences
from classiq.interface.model.model import Model as Model_Designer

import numpy as np
import json

In [9]:
from classiq import *
from classiq.qmod.symbolic import pi

# Define precision for qubits representing time steps or energy resolution
precision = 4

@qfunc
def compute_energy(precision: CInt, x: QNum, energy: Output[QNum]) -> None:
    """
    Simulates the time evolution operator by alternating potential and kinetic energy.
    precision: Controls how many fractional digits or resolution are used
    x: Position variable (QNum)
    energy: Output to store the energy value at each step
    """

    # Potential Energy Operator: Apply RZ rotation in the position basis
    RZ_phase = pi * (x**2) / 2  # proportional to x^2
    RZ(RZ_phase, x)  # Apply RZ directly to position variable `x`

    # Kinetic Energy Operator: Apply RX for momentum
    RX_phase = pi * (x**2) / 2  # proportional to x^2
    RX(RX_phase, x)  # Apply RX directly to the same qubit for kinetic energy

    # Measure energy and store in the output
    measure_energy(x, energy)  # Placeholder for measuring the energy

@qfunc
def main(x: Output[QNum], y: Output[QNum]) -> None:
    """
    Main function to initialize the system and perform time evolution for the harmonic oscillator.
    """
    # Initialize the quantum variable for position (x) with the given precision
    allocate_num(num_qubits=precision, is_signed=False, fraction_digits=precision, out=x)

    # Initialize over all possible values of the quantum number with the Hadamard transform
    hadamard_transform(x)

    # Compute the energy using the quantum time evolution operator
    compute_energy(precision, x, y)

# Classical function to simulate different energy scenarios at different time steps
@cfunc
def cmain() -> None:
    result = sample({"rotation_angle": pi / 3})
    save({"result": result})
    result = sample({"rotation_angle": pi / 2})
    save({"result": result})

# Create and synthesize the quantum model for execution
model = create_model(main, classical_execution_function=cmain)
qprog = synthesize(model)
show(qprog)

ClassiqValueError: Argument '((pi) * ((x) ** 2)) / 2' to parameter 'theta' of function 'RZ' has incompatible type; expected CReal
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack